In [2]:
import boto3
import pandas as pd


def load_data_s3(bucket_name='10academy-group2-bucket', file_name='Clean_Amharic.txt'):
    """ Load transcription data from s3 bucket"""
    s3 = boto3.resource(
        service_name='s3',
        region_name='us-east-1'
    )
    # Load file directly into python
    obj = s3.Bucket(bucket_name).Object(file_name).get()
    df = pd.read_csv(obj['Body'])
    return df

In [3]:
df = load_data_s3()

In [4]:
# Renaming the column name

In [5]:
df.columns = df.columns.str.replace("(","")


/tmp/ipykernel_15396/2007145550.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace("(","")


In [6]:
df.columns = df.columns.str.replace(")","")


/tmp/ipykernel_15396/2536619403.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(")","")


In [7]:
df.columns = df.columns.str.replace("ብርሀን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየአመቱ የሚያዘጋጀው የክለቦች ቻምፒዮና በአዲስ አበባ ከተማ በመካሄድ ላይ ይገኛል sentence 1","sentence")


In [8]:
df

,sentence
0,የተሻለ ብቃት ያሳዩ ቦክሰኞች ለቶኪዮ ኦሊምፒክ ማጣሪያ ተሳታፊ እንደሚሆ...
1,በቦክስ ስፖርት ከሚካሄዱት አመታዊ ቻምፒዮናዎች መካከል አንዱ በክለቦች መ...
2,የኢትዮጵያ ቦክስ ፌዴሬሽን ከአዲስ አበባ ቦክስ ፌዴሬሽን ጋር በመተባበር...
3,በውድድሩም ላይ አምስት ክለቦች ከአዲስ አበባ ፌዴራልፖሊስ ፌዴራል ማረሚ...
4,በአጠቃላይ ክለቦቻቸውን ወክለውም 12 ሴትና 76 ወንድ ቦክሰኞች በጥቅሉ...
...,...
662353,የ2011 በጀት አመት የውጭ ዲሎማሲያዊ ተግባራት ስኬታማ እንደነበሩ ተገለ...
662354,አቶ አገኘሁ ተሻገር የሰላም ግንባታና የህዝብ ደህንነት ሀላፊ ሆነው ተሾሙ...
662355,የአማራ ክልል ምክር ቤት የ230 ዳኞችን ሹመት አፀደቀ (sentence 6...
662356,ሀምሌ 22 ለሚካሄደው የችግኝ ተከላ መርሀግብር 54 ሚሊየን ብር ተመድቧል...


In [9]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("Spark_reading_s3") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(df) 
sparkDF.printSchema()
sparkDF.show()

root
 |-- sentence: string (nullable = true)



21/09/13 10:58:26 WARN TaskSetManager: Stage 0 contains a task of very large size (184644 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+
|            sentence|
+--------------------+
| የተሻለ ብቃት ያሳዩ ቦክሰ...|
|በቦክስ ስፖርት ከሚካሄዱት ...|
| የኢትዮጵያ ቦክስ ፌዴሬሽን...|
| በውድድሩም ላይ አምስት ክ...|
| በአጠቃላይ ክለቦቻቸውን ወ...|
| ውድድሩ የሚካሄደው በራስ ...|
| የቦክስ ስፖርት ከንክኪ ስ...|
| በመሆኑም በዚህ ረገድ ውድ...|
| ይኸውም ተወዳዳሪዎችና አሰ...|
| ከዚህ ባሻገር ውድድር የሚ...|
| ኮቪድ 19 አለም አቀፍ ወ...|
| ከእነዚህ ውድድሮች መካከል...|
| ውድድሮች ወደ እንቅስቃሴ ...|
| ኢትዮጵያም በዚህ ቻምፒዮና...|
| ቻምፒዮናው አራት ዙሮች ያ...|
| ቻምፒዮናው ጥር 42013 ...|
| የአዲስ ዘመን ጋዜጣ ቀደም...|
|እኛም ከእነዚህ ዘገባዎች ጋ...|
| በዛሬው የአዲስ ዘመን ዱሮ...|
| ማገጃ ምልክት ይሰጠውማንኛ...|
+--------------------+
only showing top 20 rows



In [10]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
#Create User defined Custom Schema using StructType
mySchema = StructType([ StructField("Id", IntegerType(), True)\
                       ,StructField("Sentence", StringType(), True)])

#Create DataFrame by changing schema
sparkDF2 = spark.createDataFrame(df,schema=mySchema)
sparkDF2.printSchema()
sparkDF2.show()

ValueError: Length of object (1) does not match with length of fields (2)